In [1]:
print("Merhaba")

Merhaba


In [1]:
from nobet.db.models import (NobetGorevi, NobetOgretmen,
                             NobetDersProgrami)
from sqlalchemy import select, and_, exists
from contextlib import contextmanager
from nobet.utils.DagitimMotoru import EnhancedNobetDagitim
from nobet.utils.rapor import create_excel_report
from nobet.db.database import SessionLocal

class DatabaseManager:
    """Handles all database operations"""
    @contextmanager
    def get_db_session(self):
        """Database session context manager"""
        session = SessionLocal()
        try:
            yield session
        except Exception:
            session.rollback()
            raise
        finally:
            session.close()

    def get_ogretmen_by_id(self, teacher_id):
        """Get teacher by ID"""
        with self.get_db_session() as session:
            return session.get(NobetOgretmen, teacher_id)

    def get_ogretmen_schedule(self, teacher_id, day):
        """Get teacher's schedule for a specific day"""
        with self.get_db_session() as session:
            stmt = select(NobetDersProgrami).where(
                (NobetDersProgrami.gun == day) &
                (NobetDersProgrami.ders_ogretmeni_id == teacher_id)
            )
            programs = session.execute(stmt).scalars().all()
            return {"ogretmen_id":teacher_id,'dersleri':{program.ders_saati: program.subeadi for program in programs}}

    def get_ogretmen_for_day(self, day):
        """Get all teachers scheduled for a specific day (without nobeti_var check)"""
        with self.get_db_session() as session:
            stmt = select(NobetOgretmen).where(
                exists().where(
                    and_(
                        NobetDersProgrami.ders_ogretmeni_id == NobetOgretmen.id,
                        NobetDersProgrami.gun == day
                    )
                )
            )
            teachers = session.execute(stmt).scalars().all()
            return {t.id: f"{t.adi_soyadi} ({t.brans})" for t in teachers}

    def get_duty_teachers(self, day):
        """Get teachers assigned to duty for a specific day"""
        with self.get_db_session() as session:
            stmt = select(NobetGorevi, NobetOgretmen).join(
                NobetOgretmen, NobetGorevi.nobetci_ogretmen_id == NobetOgretmen.id
            ).where(NobetGorevi.nobet_gun == day)
            results = session.execute(stmt).all()
            return [(nobet_gorevi, ogretmen) for nobet_gorevi, ogretmen in results]

class TeacherManager(DatabaseManager):
    """Handles teacher-specific operations, inheriting from DatabaseManager"""
    def __init__(self):
        super().__init__()
        self.teacher_id_to_name = {}

    def get_ogretmen(self,teacher_id):
        teacher = self.get_ogretmen_by_id(teacher_id)
        if teacher_id not in self.teacher_id_to_name:
            if teacher:
                self.teacher_id_to_name[teacher_id] = teacher.adi_soyadi
        return teacher

    def get_ogretmen_adi(self, teacher_id):
        """Get teacher's name by ID with caching"""
        if teacher_id not in self.teacher_id_to_name:
            teacher = self.get_ogretmen_by_id(teacher_id)
            if teacher:
                self.teacher_id_to_name[teacher_id] = teacher.adi_soyadi
        return self.teacher_id_to_name.get(teacher_id, "Unknown Teacher")

    def get_ogretmen_programi(self, teacher_id, day, ayrinti=False):
        #Get formatted schedule summary for a teacher
        schedule = self.get_ogretmen_schedule(teacher_id, day)
        if ayrinti:
            teacher = self.get_ogretmen_by_id(teacher_id)
            teacher_data = {
                    #'ogretmen_id': teacher.id,
                    'adi_soyadi': teacher.adi_soyadi,
                    'brans': teacher.brans,
                    #'dersler': schedule['dersler']
            }
            schedule.update(teacher_data)
            return schedule
            
        return schedule

    def get_gunun_nobetcileri(self, day):
        nobetciler = self.get_duty_teachers(day)
        return nobetciler

    def get_gunun_ogretmenleri(self, day):
        teacher = self.get_ogretmen_for_day(day)
        return teacher


In [2]:
ogretmen= TeacherManager()

In [3]:
secilen_gun = "Wednesday"

In [4]:
list_ogretmen = ogretmen.get_ogretmen_programi(1,"Wednesday", ayrinti=False)
print(list_ogretmen)

{'ogretmen_id': 1, 'dersleri': {6: '12 / J', 7: '12 / J', 8: '12 / J'}}


In [19]:
nobetciler_pts = ogretmen.get_gunun_ogretmenleri(secilen_gun)
for nobetci, teacher in nobetciler_pts.items():
    print(f"Nöbet Yeri: {nobetci}")
    print(f"Öğretmen: {teacher}")
    print("-" * 30)
print(len(nobetciler_pts))

Nöbet Yeri: 1
Öğretmen: ABDULKADİR KARAYEL (İngilizce)
------------------------------
Nöbet Yeri: 2
Öğretmen: AHMET GÜREL (Müzik)
------------------------------
Nöbet Yeri: 4
Öğretmen: ALİ ÇELİK (Coğrafya)
------------------------------
Nöbet Yeri: 5
Öğretmen: ALİ KAYGISIZ (Matematik)
------------------------------
Nöbet Yeri: 6
Öğretmen: ALİ KURUOĞLU (Felsefe)
------------------------------
Nöbet Yeri: 7
Öğretmen: ALMİRA TUĞBA KAL (Görsel Sanatlar)
------------------------------
Nöbet Yeri: 10
Öğretmen: AYSAN KESKİN (İngilizce)
------------------------------
Nöbet Yeri: 11
Öğretmen: AYSUN BESLER (Biyoloji)
------------------------------
Nöbet Yeri: 12
Öğretmen: AYSUN UĞUR (Görsel Sanatlar)
------------------------------
Nöbet Yeri: 14
Öğretmen: AYŞE TURHAN (Almanca)
------------------------------
Nöbet Yeri: 15
Öğretmen: AYŞEGÜL ÇELİK (İngilizce)
------------------------------
Nöbet Yeri: 16
Öğretmen: BAŞAK BAKAR (Matematik)
------------------------------
Nöbet Yeri: 17
Öğretmen: BEDR

In [82]:
adi

In [83]:
dersler = teacher_manager.get_teacher_schedule(1,"Wednesday")
print(dersler)

{'ogretmen_id': 1, 'dersler': {6: '12 / J', 7: '12 / J', 8: '12 / J'}}


In [111]:
#Hata Yönetimli Yeni Sınıf

In [114]:
from sqlalchemy.exc import SQLAlchemyError
from typing import Optional, Dict, List, Tuple, Union
from nobet.db.models import (NobetGorevi, NobetOgretmen,
                             NobetDersProgrami)
from sqlalchemy import select, and_, exists
from contextlib import contextmanager
from nobet.utils.DagitimMotoru import EnhancedNobetDagitim
from nobet.utils.rapor import create_excel_report
from nobet.db.database import SessionLocal

class DatabaseManager:
    """Handles all database operations with comprehensive error handling"""
    
    @contextmanager
    def get_db_session(self):
        """Database session context manager with error handling"""
        session = SessionLocal()
        try:
            yield session
            session.commit()
        except SQLAlchemyError as e:
            session.rollback()
            raise DatabaseError(f"Database operation failed: {str(e)}") from e
        except Exception as e:
            session.rollback()
            raise DatabaseError(f"Unexpected error: {str(e)}") from e
        finally:
            session.close()

    def get_ogretmen_by_id(self, teacher_id: int) -> Optional[NobetOgretmen]:
        """Get teacher by ID with validation"""
        if not isinstance(teacher_id, int) or teacher_id <= 0:
            raise ValueError("Geçersiz öğretmen ID'si")
            
        try:
            with self.get_db_session() as session:
                teacher = session.get(NobetOgretmen, teacher_id)
                if not teacher:
                    raise NotFoundError(f"{teacher_id} ID'li öğretmen bulunamadı")
                return teacher
        except SQLAlchemyError as e:
            raise DatabaseError(f"Öğretmen bilgisi alınamadı: {str(e)}")

    def get_ogretmen_schedule(self, teacher_id: int, day: str) -> Dict[str, Union[int, Dict]]:
        """Get teacher's schedule with validation"""
        if not isinstance(teacher_id, int) or teacher_id <= 0:
            raise ValueError("Geçersiz öğretmen ID'si")
        if not day or not isinstance(day, str):
            raise ValueError("Geçersiz gün bilgisi")

        try:
            with self.get_db_session() as session:
                stmt = select(NobetDersProgrami).where(
                    (NobetDersProgrami.gun == day) &
                    (NobetDersProgrami.ders_ogretmeni_id == teacher_id)
                )
                programs = session.execute(stmt).scalars().all()
                
                if not programs:
                    raise NotFoundError(f"{teacher_id} ID'li öğretmenin {day} günü dersi bulunamadı")
                    
                return {
                    "ogretmen_id": teacher_id,
                    'dersler': {program.ders_saati: program.subeadi for program in programs}
                }
        except SQLAlchemyError as e:
            raise DatabaseError(f"Ders programı alınamadı: {str(e)}")

    def get_ogretmen_for_day(self, day: str) -> Dict[int, str]:
        """Get all teachers scheduled for a specific day with validation"""
        if not day or not isinstance(day, str):
            raise ValueError("Geçersiz gün bilgisi")

        try:
            with self.get_db_session() as session:
                stmt = select(NobetOgretmen).where(
                    exists().where(
                        and_(
                            NobetDersProgrami.ders_ogretmeni_id == NobetOgretmen.id,
                            NobetDersProgrami.gun == day
                        )
                    )
                )
                teachers = session.execute(stmt).scalars().all()
                
                if not teachers:
                    raise NotFoundError(f"{day} günü için kayıtlı öğretmen bulunamadı")
                    
                return {t.id: f"{t.adi_soyadi} ({t.brans})" for t in teachers}
        except SQLAlchemyError as e:
            raise DatabaseError(f"Öğretmen listesi alınamadı: {str(e)}")

    def get_duty_teachers(self, day: str) -> List[Tuple[NobetGorevi, NobetOgretmen]]:
        """Get teachers assigned to duty with validation"""
        if not day or not isinstance(day, str):
            raise ValueError("Geçersiz gün bilgisi")

        try:
            with self.get_db_session() as session:
                stmt = select(NobetGorevi, NobetOgretmen).join(
                    NobetOgretmen, NobetGorevi.nobetci_ogretmen_id == NobetOgretmen.id
                ).where(NobetGorevi.nobet_gun == day)
                
                results = session.execute(stmt).all()
                
                if not results:
                    raise NotFoundError(f"{day} günü için nöbetçi ataması bulunamadı")
                    
                return [(nobet_gorevi, ogretmen) for nobet_gorevi, ogretmen in results]
        except SQLAlchemyError as e:
            raise DatabaseError(f"Nöbetçi listesi alınamadı: {str(e)}")


class DatabaseError(Exception):
    """Custom database exception"""
    pass

class NotFoundError(Exception):
    """Custom not found exception"""
    pass

class TeacherManager(DatabaseManager):
    """Handles teacher-specific operations, inheriting from DatabaseManager"""
    def __init__(self):
        super().__init__()
        self.teacher_id_to_name = {}

    def get_ogretmen(self,teacher_id):
        teacher = self.get_ogretmen_by_id(teacher_id)
        if teacher_id not in self.teacher_id_to_name:
            if teacher:
                self.teacher_id_to_name[teacher_id] = teacher.adi_soyadi
        return teacher

    def get_ogretmen_adi(self, teacher_id):
        """Get teacher's name by ID with caching"""
        if teacher_id not in self.teacher_id_to_name:
            teacher = self.get_ogretmen_by_id(teacher_id)
            if teacher:
                self.teacher_id_to_name[teacher_id] = teacher.adi_soyadi
        return self.teacher_id_to_name.get(teacher_id, "Unknown Teacher")

    def get_ogretmen_programi(self, teacher_id, day, ayrinti=False):
        #Get formatted schedule summary for a teacher
        schedule = self.get_ogretmen_schedule(teacher_id, day)
        if ayrinti:
            teacher = self.get_ogretmen_by_id(teacher_id)
            teacher_data = {
                    #'ogretmen_id': teacher.id,
                    'adi_soyadi': teacher.adi_soyadi,
                    'brans': teacher.brans,
                    #'dersler': schedule['dersler']
            }
            schedule.update(teacher_data)
            return schedule
            
        return schedule

    def get_gunun_nobetcileri(self, day):
        nobetciler = self.get_duty_teachers(day)
        return nobetciler

In [116]:
ogretmen= TeacherManager()
secilen_gun = "Monday"

In [6]:
class SimplifiedNobetDagitim:
    def __init__(self):
        self.penalty_weights = {
            'overload': 100,      # Max 3 duties per teacher
            'inequality': 10,     # Uneven distribution
            'unassigned': 1000,   # Unassigned classes
            'stats_diff': 5,      # Deviation from historical average
            'conflict': 1000     # Schedule conflicts
        }
        self.temp_stats = {}      # Temporary statistics storage
    
    def calculate_penalty(self, assignment):
        """Calculate penalty for the current assignment"""
        penalty = 0
        nobetci_counts = assignment['teacher_counts']
        
        # 1. Overload penalty (max 3 duties)
        for count in nobetci_counts.values():
            if count > 3:
                penalty += (count - 3) * self.penalty_weights['overload']
        
        # 2. Inequality penalty
        if nobetci_counts:
            penalty += (max(nobetci_counts.values()) - min(nobetci_counts.values())) * self.penalty_weights['inequality']
        
        # 3. Unassigned classes penalty
        penalty += len(assignment['unassigned']) * self.penalty_weights['unassigned']
        
        # 4. Conflict penalty
        penalty += self.check_conflicts(assignment) * self.penalty_weights['conflict']
        
        return penalty
    
    def check_conflicts(self, assignment):
        """Check for scheduling conflicts"""
        teacher_schedule = {}
        conflicts = 0
        
        for assignment in assignment['assignments']:
            teacher_id = assignment['teacher_id']
            hour = assignment['hour']
            
            if teacher_id not in teacher_schedule:
                teacher_schedule[teacher_id] = set()
            
            if hour in teacher_schedule[teacher_id]:
                conflicts += 1
            else:
                teacher_schedule[teacher_id].add(hour)
        
        return conflicts
    
    def optimize(self, available_teachers, absent_teachers):
        """Optimize duty assignments"""
        # Prepare availability data
        availability = self.prepare_availability(available_teachers)
        absent_classes = self.flatten_absent(absent_teachers)
        
        # Generate initial solution
        solution = self.initial_solution(available_teachers, availability, absent_classes)
        best_penalty = self.calculate_penalty(solution)
        
        # Simple optimization - try to assign unassigned classes
        for unassigned in solution['unassigned'][:]:
            for teacher in available_teachers:
                teacher_id = teacher['ogretmen_id']
                hour = unassigned['hour']
                class_name = unassigned['class']
                
                if (availability[teacher_id][hour] and 
                    solution['teacher_counts'].get(teacher_id, 0) < 3 and
                    not self.has_conflict(solution['assignments'], teacher_id, hour)):
                    
                    # Assign this class
                    solution['assignments'].append({
                        'hour': hour,
                        'class': class_name,
                        'teacher_id': teacher_id,
                        'absent_teacher_id': unassigned['absent_teacher_id']
                    })
                    solution['teacher_counts'][teacher_id] = solution['teacher_counts'].get(teacher_id, 0) + 1
                    solution['unassigned'].remove(unassigned)
                    break
        
        return solution
    
    def prepare_availability(self, teachers):
        """Prepare teacher availability dictionary"""
        availability = {}
        for teacher in teachers:
            teacher_id = teacher['ogretmen_id']
            busy_hours = set(teacher['dersleri'].keys())
            availability[teacher_id] = {
                hour: (hour not in busy_hours) 
                for hour in range(1, 9)  # Assuming 8 hours in a day
            }
        return availability
    
    def flatten_absent(self, absent_teachers):
        """Flatten absent teachers' schedule"""
        absent_classes = []
        for teacher in absent_teachers:
            for hour, class_name in teacher['dersleri'].items():
                absent_classes.append({
                    'hour': hour,
                    'class': class_name,
                    'absent_teacher_id': teacher['ogretmen']
                })
        return absent_classes
    
    def initial_solution(self, teachers, availability, absent_classes):
        """Create initial solution"""
        solution = {
            'assignments': [],
            'unassigned': [],
            'teacher_counts': {}
        }
        
        for class_info in absent_classes:
            assigned = False
            hour = class_info['hour']
            
            for teacher in teachers:
                teacher_id = teacher['ogretmen_id']
                
                if (availability[teacher_id][hour] and 
                    solution['teacher_counts'].get(teacher_id, 0) < 3):
                    
                    solution['assignments'].append({
                        'hour': hour,
                        'class': class_info['class'],
                        'teacher_id': teacher_id,
                        'absent_teacher_id': class_info['absent_teacher_id']
                    })
                    solution['teacher_counts'][teacher_id] = solution['teacher_counts'].get(teacher_id, 0) + 1
                    assigned = True
                    break
            
            if not assigned:
                solution['unassigned'].append(class_info)
        
        return solution
    
    def has_conflict(self, assignments, teacher_id, hour):
        """Check if teacher has conflict at given hour"""
        return any(
            a['teacher_id'] == teacher_id and a['hour'] == hour
            for a in assignments
        )

# Usage example:
nobetci_dersleri = [
    {'ogretmen_id': 45, 'dersleri': {2: '12 / F', 7: '12 / G', 8: '12 / G'}},
    {'ogretmen_id': 69, 'dersleri': {1: '9 / F', 2: '9 / F', 7: '9 / K'}},
    # ... other teachers ...
]

devamsiz_dersleri = [
    {'ogretmen': 74, 'dersleri': {1: '10 / A', 8: '9 / J'}}
]

distributor = SimplifiedNobetDagitim()
solution = distributor.optimize(nobetci_dersleri, devamsiz_dersleri)

print("Assignments:", solution['assignments'])
print("Unassigned classes:", solution['unassigned'])
print("Teacher duty counts:", solution['teacher_counts'])

Assignments: [{'hour': 1, 'class': '10 / A', 'teacher_id': 45, 'absent_teacher_id': 74}, {'hour': 8, 'class': '9 / J', 'teacher_id': 69, 'absent_teacher_id': 74}]
Unassigned classes: []
Teacher duty counts: {45: 1, 69: 1}


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from sklearn.preprocessing import StandardScaler
from collections import defaultdict
import random

class AINobetDagitim:
    def __init__(self, population_size=100, generations=200, mutation_rate=0.15):
        # Ceza ağırlıkları (öğrenilebilir parametreler haline gelecek)
        self.penalty_weights = tf.Variable({
            'overload': tf.random.uniform([], 800, 1200),
            'inequality': tf.random.uniform([], 700, 1100),
            'unassigned': tf.random.uniform([], 900, 1300),
            'conflict': tf.random.uniform([], 9500, 10500),
            'no_duty': tf.random.uniform([], 4500, 5500)
        }, dtype=tf.float32)
        
        # Yapay zeka modeli
        self.model = self.build_model()
        self.yuksiniri = 2
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.scaler = StandardScaler()

    def build_model(self):
        """Öğretmen ve ders özelliklerini işlemek için sinir ağı modeli"""
        model = models.Sequential([
            layers.Dense(64, activation='relu', input_shape=(10,)),  # 10 özellik
            layers.Dropout(0.2),
            layers.Dense(32, activation='relu'),
            layers.Dense(1, activation='sigmoid')  # Uygunluk skoru
        ])
        
        model.compile(optimizer='adam',
                     loss='binary_crossentropy',
                     metrics=['accuracy'])
        return model

    def create_individual(self):
        """Yapay zeka destekli birey oluşturma"""
        solution = {
            'assignments': [],
            'teacher_counts': defaultdict(int),
            'teacher_schedule': defaultdict(set)
        }
        
        # Dersleri öncelik sırasına göre sırala (AI tahmini)
        classes = self.prioritize_classes(self.absent_classes)
        
        # Öğretmenleri uygunluk skoruna göre sırala
        teachers_sorted = self.rank_teachers(classes)
        
        # Atama yap
        for class_info in classes:
            hour = class_info['hour']
            best_teacher = self.find_best_teacher(hour, teachers_sorted, solution)
            
            if best_teacher:
                solution['assignments'].append({
                    'hour': hour,
                    'class': class_info['class'],
                    'teacher_id': best_teacher,
                    'absent_teacher_id': class_info['absent_teacher_id']
                })
                solution['teacher_counts'][best_teacher] += 1
                solution['teacher_schedule'][best_teacher].add(hour)
        
        return solution

    def prioritize_classes(self, classes):
        """Dersleri önem sırasına göre sıralar"""
        # Her ders için özellik vektörü oluştur
        features = []
        for c in classes:
            features.append([
                c['hour'],  # Saat
                len(self.find_available_teachers(c['hour'])),  # Uygun öğretmen sayısı
                # Diğer özellikler...
            ])
        
        # Ölçeklendirme
        if len(features) > 1:
            features = self.scaler.fit_transform(features)
        
        # Öncelik skorlarını tahmin et
        scores = self.model.predict(np.array(features)).flatten()
        
        # Skorlara göre sırala
        sorted_indices = np.argsort(scores)[::-1]  # Yüksek skor önce
        return [classes[i] for i in sorted_indices]

    def train_model(self, X_train, y_train, epochs=50):
        """Modeli geçmiş verilerle eğitir"""
        # Veri ön işleme
        X_train = self.scaler.fit_transform(X_train)
        
        # Eğitim
        history = self.model.fit(
            X_train, y_train,
            epochs=epochs,
            batch_size=32,
            validation_split=0.2,
            verbose=1
        )
        
        return history

    def optimize(self, available_teachers, absent_teachers):
        """Yapay zeka destekli optimizasyon"""
        # Veri hazırlığı
        self.availability = self.prepare_availability(available_teachers)
        self.absent_classes = self.flatten_absent(absent_teachers)
        self.teachers = available_teachers
        self.teacher_ids = [t['ogretmen_id'] for t in available_teachers]
        
        # Özellik mühendisliği
        self.prepare_features()
        
        # Başlangıç popülasyonu (AI destekli)
        population = [self.create_individual() for _ in range(self.population_size)]
        
        # Genetik algoritma
        for generation in range(self.generations):
            # Çözümleri değerlendir
            evaluated = [(self.calculate_penalty(ind), ind) for ind in population]
            evaluated.sort(key=lambda x: x[0])
            
            # En iyi çözümleri kaydet (aktif öğrenme için)
            self.record_best_solutions(evaluated[:5])
            
            # Yeni nesil
            new_population = [evaluated[0][1]]  # Elitizm
            
            while len(new_population) < self.population_size:
                # Yapay zeka destekli ebeveyn seçimi
                parent1, parent2 = self.ai_parent_selection(population)
                
                # Çaprazlama
                child = self.crossover(parent1, parent2)
                
                # Akıllı mutasyon
                child = self.smart_mutate(child)
                
                new_population.append(child)
            
            population = new_population
        
        return self.format_solution(min(population, key=lambda x: self.calculate_penalty(x)))

    def smart_mutate(self, individual):
        """Öğrenilmiş bilgiye dayalı mutasyon"""
        if random.random() < self.mutation_rate:
            # Çakışmaları çöz
            self.resolve_conflicts(individual)
            
            # Öğretmen yüklerini dengele
            self.balance_teacher_load(individual)
            
            # AI ile en iyi değişikliği seç
            self.ai_guided_mutation(individual)
        
        return individual
    
    def ai_parent_selection(self, population):
        """Model tahminlerine göre ebeveyn seçer"""
        # Çözümleri değerlendir
        scores = [self.model_evaluate(ind) for ind in population]
        
        # Olasılıkları hesapla
        probs = tf.nn.softmax(scores).numpy()
        
        # Olasılıklara göre ebeveyn seç
        parent1, parent2 = np.random.choice(population, size=2, p=probs, replace=False)
        return parent1, parent2

# Örnek kullanım
if __name__ == "__main__":
    # Eğitim verilerini yükle
    X_train, y_train = load_training_data("nobet_verileri.csv")
    
    # Sistem oluştur
    nobet_sistemi = AINobetDagitim()
    
    # Modeli eğit
    nobet_sistemi.train_model(X_train, y_train, epochs=100)
    
    # Örnek veri
    available_teachers = [...]  # Müsait öğretmenler
    absent_teachers = [...]     # Devamsız öğretmenler
    
    # Optimizasyon yap
    solution = nobet_sistemi.optimize(available_teachers, absent_teachers)
    
    # Sonuçları görüntüle
    print("En iyi çözüm:")
    print(f"Toplam ceza: {solution['penalty']}")
    print(f"Atanan nöbetler: {len(solution['assignments']}")
    print(f"Atanamayan dersler: {len(solution['unassigned'])}")

In [2]:
class AdvancedNobetDagitim:
    def __init__(self, population_size=50, generations=100, mutation_rate=0.1):
        self.penalty_weights = {'overload': 100, 'inequality': 100, 'unassigned': 10, 'no_duty': 500}
        self.yuksiniri = 3
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
    
    def optimize(self, available_teachers, absent_teachers):
        self.availability = self.prepare_availability(available_teachers)
        self.absent_classes = self.flatten_absent(absent_teachers)
        self.teachers = available_teachers
        self.teacher_ids = [t['ogretmen_id'] for t in available_teachers]
        
        # Başlangıç popülasyonu
        population = [self.create_individual() for _ in range(self.population_size)]
        
        for _ in range(self.generations):
            population.sort(key=lambda x: self.calculate_penalty(x))
            new_population = [population[0]]

            while len(new_population) < self.population_size:
                child = self.mutate(self.create_individual())
                new_population.append(child)

            population = new_population
        
        best_solution = min(population, key=lambda x: self.calculate_penalty(x))
        return self.format_solution(best_solution)
    
    def create_individual(self):
        solution = {'assignments': [], 'teacher_counts': defaultdict(int), 'teacher_schedule': defaultdict(set)}
        for class_info in self.absent_classes:
            eligible_teachers = [t_id for t_id in self.teacher_ids if self.availability[t_id][class_info['hour']]]
            if eligible_teachers:
                teacher_id = random.choice(eligible_teachers)
                solution['assignments'].append({
                    'hour': class_info['hour'],
                    'class': class_info['class'],
                    'teacher_id': teacher_id,
                    'absent_teacher_id': class_info['absent_teacher_id']
                })
                solution['teacher_counts'][teacher_id] += 1
                solution['teacher_schedule'][teacher_id].add(class_info['hour'])
        return solution

    def mutate(self, individual):
        """ Bireyin mutasyon geçirmesini sağlar. """
        for assignment in individual['assignments']:
            if random.random() < self.mutation_rate:
                eligible_teachers = [
                    t_id for t_id in self.teacher_ids 
                    if self.availability[t_id][assignment['hour']]
                ]
                if eligible_teachers:
                    new_teacher = random.choice(eligible_teachers)
                    assignment['teacher_id'] = new_teacher
        return individual

    def calculate_penalty(self, solution):
        penalty = 0
        counts = list(solution['teacher_counts'].values())
        if counts:
            penalty += (max(counts) - min(counts)) * self.penalty_weights['inequality']
        penalty += len(self.absent_classes) - len(solution['assignments'])
        penalty += len([t_id for t_id in self.teacher_ids if solution['teacher_counts'][t_id] == 0]) * self.penalty_weights['no_duty']
        return penalty

    def format_solution(self, solution):
        return {
            'assignments': solution['assignments'],
            'teacher_counts': dict(solution['teacher_counts']),
            'penalty': self.calculate_penalty(solution)
        }

    def prepare_availability(self, teachers):
        availability = {}
        for teacher in teachers:
            teacher_id = teacher['ogretmen_id']
            busy_hours = set(teacher['dersleri'].keys())
            availability[teacher_id] = {hour: (hour not in busy_hours) for hour in range(1, 9)}
        return availability
    
    def flatten_absent(self, absent_teachers):
        return [{'hour': int(h), 'class': c, 'absent_teacher_id': t['ogretmen_id']} 
                for t in absent_teachers for h, c in t['dersleri'].items()]


{'assignments': [{'hour': 4,
   'class': '10 / A',
   'teacher_id': 72,
   'absent_teacher_id': 66},
  {'hour': 5, 'class': '10 / E', 'teacher_id': 28, 'absent_teacher_id': 66},
  {'hour': 6, 'class': '10 / B', 'teacher_id': 17, 'absent_teacher_id': 66},
  {'hour': 7, 'class': '10 / C', 'teacher_id': 52, 'absent_teacher_id': 66},
  {'hour': 8, 'class': '10 / C', 'teacher_id': 47, 'absent_teacher_id': 66}],
 'teacher_counts': {72: 1,
  28: 1,
  17: 1,
  52: 1,
  47: 1,
  26: 0,
  14: 0,
  36: 0,
  42: 0,
  16: 0},
 'penalty': 2600}

In [1]:
def fonk(atemp:int=4):
    print(f"atemp={atemp} ve {atemp*2}")
    

In [2]:
fonk()

atemp=4 ve 8


In [3]:
fonk(5)

atemp=5 ve 10


In [4]:
fonk(atemp=10)

atemp=10 ve 20
